In [21]:
import csv
import itertools
import re
import pandas as pd


#* ENVS
wspr_csv = "2022-01-10-flt8.csv"
time_slot = 8
re_time_slot = f'^..:.{time_slot}'
parsed_csv = f'parsed_{wspr_csv}'

header = []
parsed_dt = []
alt = []


def alt_calc(power,split_ts):
    if split_ts == None:
        alt_m = int(power)*20 #* Type 3 WSPR message decode x 20 (20m per dBm)
    else:
        alt_m = int(power)*300 #* Type 1 or 2 WSPR message decode x 300 (300m per dBm)
    alt_ft = alt_m * 3.28 #* Convert meters to feet
    return (alt_m, int(alt_ft))


with open(wspr_csv, 'r') as file:
    data = csv.reader(file)
    header = next(data)
    for row in data:
        alt = []
        split_dt = row[1].split(' ')
        split_ts = re.search(re_time_slot, split_dt[1])
        alt_m = alt_calc(row[15], split_ts)
        alt.extend(alt_m)
        parsed_dt.append(list(itertools.chain(row, split_dt, alt)))

add_headers = ['date', 'time', 'altitude_m', 'altitude_ft']
header.extend(add_headers)

with open(parsed_csv, "w",newline="") as f:
    writer = csv.writer(f)
    header[1] = 'datetime'
    # print(header)
    writer.writerow(header)
    for row in parsed_dt:
        writer.writerow(row)

In [22]:
#* Adding in a dataframe to work to add the WSPR 3 packet to the Type 1 and 2

parsed_df = pd.read_csv(parsed_csv, parse_dates=["datetime"], usecols=['datetime','date','time', 'tx_lat', 'tx_lon', 'tx_loc', 'altitude_m', 'altitude_ft'])
parsed_df.tail()

,datetime,tx_lat,tx_lon,tx_loc,date,time,altitude_m,altitude_ft
4084,2022-01-10 23:38:00,7.771,158.792,QJ97js,2022-01-10,23:38:00,8100,26568
4085,2022-01-10 23:38:00,7.771,158.792,QJ97js,2022-01-10,23:38:00,8100,26568
4086,2022-01-10 23:38:00,7.771,158.792,QJ97js,2022-01-10,23:38:00,8100,26568
4087,2022-01-10 23:38:00,7.771,158.792,QJ97js,2022-01-10,23:38:00,8100,26568
4088,2022-01-11 00:18:00,7.771,158.792,QJ97js,2022-01-11,00:18:00,8100,26568


In [183]:
parsed_df.dtypes

datetime       datetime64[ns]
tx_lat                float64
tx_lon                float64
tx_loc                 object
date                   object
time                   object
altitude_m              int64
altitude_ft             int64
dtype: object

In [48]:
parsed_df['type'] = parsed_df['time'].str[4]

In [49]:
parsed_df.drop_duplicates(subset ="datetime", keep = False, inplace = True)
parsed_df.shape

(329, 9)

In [16]:
parsed_df

,datetime,tx_lat,tx_lon,tx_loc,date,time,altitude_m,altitude_ft,type
7,2021-12-22 18:18:00,41.479,-87.042,EN61,2021-12-22,18:18:00,3000,9840,8
124,2021-12-22 18:58:00,41.688,-86.792,EN61oq,2021-12-22,18:58:00,5100,16728,8
125,2021-12-22 19:00:00,41.563,-86.625,EN61qn,2021-12-22,19:00:00,740,2427,0
262,2021-12-22 20:08:00,40.479,-85.042,EN70,2021-12-22,20:08:00,6900,22632,8
353,2021-12-23 12:38:00,36.479,-73.042,FM36,2021-12-23,12:38:00,6900,22632,8
...,...,...,...,...,...,...,...,...,...
4080,2022-01-10 22:38:00,7.729,159.125,QJ97nr,2022-01-10,22:38:00,6900,22632,8
4081,2022-01-10 23:00:00,7.771,158.875,QJ97ks,2022-01-10,23:00:00,1140,3739,0
4082,2022-01-10 23:18:00,7.771,158.875,QJ97ks,2022-01-10,23:18:00,8100,26568,8
4083,2022-01-10 23:20:00,7.771,158.792,QJ97js,2022-01-10,23:20:00,0,0,0


In [42]:
parsed_df.to_csv('new_wspr.csv', index = False)

In [53]:
# I only print for type 0 packets and add the 0 detail to type 8 base. 
with open("new_wspr.csv", "r") as a_file:
    for line in a_file:
        spline = line.split(',')
        # print(spline[8].strip('\n'))
        if spline[8].strip('\n') == '8':
            # print("it's an 8")
            base_altitude = int(spline[7])
        elif spline[8].strip('\n') == '0':
            altitude = base_altitude + int(spline[7])
            print(f"{spline[4]} {altitude} ft")

2021-12-22 19155 ft
2021-12-23 24796 ft
2021-12-23 24796 ft
2021-12-23 24403 ft
2021-12-24 24599 ft
2021-12-24 24403 ft
2021-12-24 24403 ft
2021-12-24 24599 ft
2021-12-24 24599 ft
2021-12-24 24599 ft
2021-12-24 24599 ft
2021-12-25 25059 ft
2021-12-25 24599 ft
2021-12-25 24796 ft
2021-12-25 25059 ft
2021-12-25 25059 ft
2021-12-25 24796 ft
2021-12-25 24796 ft
2021-12-25 25059 ft
2021-12-25 25059 ft
2021-12-25 24599 ft
2021-12-25 24796 ft
2021-12-25 24796 ft
2021-12-25 24796 ft
2021-12-25 25059 ft
2021-12-25 25452 ft
2021-12-26 25059 ft
2021-12-26 25912 ft
2021-12-26 25715 ft
2021-12-27 25715 ft
2021-12-27 25059 ft
2021-12-28 22632 ft
2021-12-28 22632 ft
2021-12-29 25452 ft
2021-12-29 25452 ft
2021-12-29 25452 ft
2021-12-29 25715 ft
2021-12-29 25715 ft
2021-12-29 25715 ft
2021-12-30 26108 ft
2021-12-30 25715 ft
2021-12-30 25912 ft
2021-12-30 25912 ft
2021-12-30 25912 ft
2021-12-30 25912 ft
2021-12-30 26108 ft
2021-12-30 26108 ft
2021-12-31 26371 ft
2021-12-31 26371 ft
2021-12-31 26371 ft


Now want to take the altitude in the :08 mins and add to the following :00 mins. to get teh precise altitude.

```
date,time,altitude_m,altitude_ft
2022-01-07,23:38:00,6900,22632
2022-01-07,23:40:00,1060,3476
```

For example the precise altitude is `7960 m`